In [ ]:
# few-shot train and test dataset construction
def read_classname(file):
    class_name = []
    with open(file, 'r') as file:
        lines = file.readlines()
        for line in lines:
            class_name.append(line.strip('\n'))
    return class_name
def read_jsonl(file,train=False,test=False):
    data_file_path = file
    f = open(data_file_path, "r", encoding="utf-8")
    data_file_list = [json.loads(line) for line in f]
    if train:
        data_file_list = [item for item in data_file_list if item["split"] == "train"]
    if test:
        data_file_list = [item for item in data_file_list if item["split"] == "valid" or item["split"] == 'test']
    return data_file_list

def choice_list(choice_n, item, total_class_list):
    random.shuffle(total_class_list)
    choice_list = total_class_list[:choice_n]
    if item['label'] not in choice_list:
        choice_list = choice_list[:-1]+[item['label']]
    random.shuffle(choice_list)

    additional_text = ' Please choose one from list [ '
    for name in choice_list:
        additional_text = additional_text + name + ', '
    additional_text = additional_text[:-2].strip() + '].'
    return additional_text

In [ ]:
import random
import json
# imagenet, Caltech101,DescribableTextures, EuroSAT, Food101, OxfordFlowers, OxfordPets, StanfordCars, SUN397, UCF101, FGVCAircraft
dataset_list = 'Imagenet, Caltech101,DescribableTextures, EuroSAT, Food101, OxfordFlowers, OxfordPets, StanfordCars, SUN397, UCF101, FGVCAircraft'
dataset_list = dataset_list.split(',')
dataset_list = [i.strip() for i in dataset_list]
dataset = dataset_list[0]
print(dataset)
shot = 4
setting = 'few-shot' 
assert setting=='b2n' or setting=='few-shot'
assert dataset in dataset_list

In [ ]:
# imagenet has classes with same name
if dataset=='Imagenet':
    file = 'classnames.txt'
    class_name = []
    with open('class_list/all_datasets/'+file, 'r') as file:
        lines = file.readlines()
        for line in lines:
            prefix = line.split(' ')[0]
            class_name.append(line[len(prefix):].strip().strip('\n'))
else:
    class_name = read_classname('class_list/all_datasets/'+dataset+'_classname.txt')
data_file_list_test = read_jsonl('class_list/all_datasets/'+dataset+'.jsonl',test=True)
data_file_list_train = read_jsonl('class_list/all_datasets/'+dataset+'.jsonl',train=True)

In [ ]:
print(data_file_list_train[0])
print(len(data_file_list_train))
print(len(data_file_list_test))
print(len(class_name))

In [ ]:
choice_n = int(len(class_name)*0.4)
if choice_n<30:
    if len(class_name)<30:
        choice_n=len(class_name)
    else:
        choice_n=len(class_name)
if choice_n>100:
    choice_n=100
print(choice_n)

In [ ]:
num_map = {}
for i in range(len(class_name)):
    if class_name[i] in num_map.keys():
        num_map[class_name[i]] += shot
    else:
        num_map[class_name[i]] = shot

In [ ]:
#What type of object is in the centered satellite photo?
#What actions is the person performing in the photo?
#What type of texture is in the photo?
#What type of aircraft is in the photo?
# 'Imagenet, Caltech101,DescribableTextures, EuroSAT, Food101, OxfordFlowers, OxfordPets, StanfordCars, SUN397, UCF101, FGVCAircraft'
if dataset=='EuroSAT':
    print(dataset)
    prefix = 'What type of object is in the centered satellite photo?'
elif dataset=='UCF101':
    print(dataset)
    prefix = 'What actions is the person performing in the photo?'
elif dataset=='DescribableTextures':
    print(dataset)
    prefix = 'What type of texture is in the photo?'
elif dataset=='FGVCAircraft':
    print(dataset)
    prefix = 'What type of aircraft is in the photo?'
else:
    print(dataset)
    prefix = 'What type of object is in the photo?'
    
random.shuffle(data_file_list_train)
fewshot_lines_train = []
for i in range(len(data_file_list_train)):
    item = data_file_list_train[i]
    if num_map[item['label']]>0:
        choice_text = choice_list(choice_n, item, class_name)
        item['problem'] = prefix+'\n'+choice_text
        fewshot_lines_train.append(item)
        num_map[item['label']] -= 1
    if len(fewshot_lines_train) == shot*len(class_name):
        break
        
test_lines = []
for item in data_file_list_test:
    choice_text = choice_list(choice_n, item, class_name)
    item['problem'] = prefix+'\n'+choice_text
    test_lines.append(item)
   

In [ ]:
print(fewshot_lines_train[0])
print(test_lines[0])

In [ ]:
for i in num_map.values():
    assert i==0

In [ ]:
# check whether lines are correct
def parse_class(line):
    line = line.split('[')[1][:-2]
    classes = line.split(',')
    classes = [i.strip() for i in classes]
    return classes

num_map = {}
for i in range(len(class_name)):
    num_map[class_name[i]] = 0
    
c_class = []
for item in fewshot_lines_train:
    c_class.append(item['label'])
    num_map[item['label']] += 1
    assert item['label'] in class_name
    choice_text = parse_class(item['problem'])
    assert item['label'] in choice_text
    assert len(choice_text)==choice_n
    for c in choice_text:
        if c not in class_name:
            print('error 1',c)
            exit()
print(len(set(c_class)),len(class_name))
assert len(set(c_class))==len(class_name)
for i in num_map.values():
    assert i==shot
assert len(fewshot_lines_train) == shot*len(class_name)
print(len(test_lines))
for item in test_lines:
    assert item['label'] in class_name
    choice_text = parse_class(item['problem'])
    assert item['label'] in choice_text
    for c in choice_text:
        if c not in class_name:
            print('error 2',c)


In [ ]:
print(len(fewshot_lines_train))

In [ ]:
prefix = '/Users/mingli/Desktop/class_list/datasets_fewshot_mc_40percent/'
with open(prefix+dataset+'_fewshot_'+str(shot)+".jsonl", 'w') as file:
    for item in fewshot_lines_train:
        # Write each dictionary as a JSON object on a new line
        json.dump(item, file)
        file.write('\n')


'''with open(prefix+dataset+'_fewshot_'+"test.jsonl", "w") as file:
    for item in test_lines:
        # Write each dictionary as a JSON object on a new line
        json.dump(item, file)
        file.write('\n')'''

